# Демонстрация LLaVA: Крупная языковая и визуальная модель

Этот ноутбук демонстрирует работу модели LLaVA (Large Language and Vision Assistant), представленной на конференции NeurIPS 2023. LLaVA объединяет возможности обработки изображений и текста, используя архитектуру, состоящую из визуального энкодера CLIP, проекционного слоя и языковой модели Vicuna.

## Основные компоненты LLaVA:
1. **Визуальный энкодер CLIP-ViT-L/14** для преобразования изображений в векторные представления
2. **Проекционный слой**, отображающий визуальные эмбеддинги в пространство языковой модели
3. **Языковая модель Vicuna** (на базе LLaMA) с 13B параметров, отвечающая за генерацию текста

Давайте установим необходимые зависимости и загрузим модель для демонстрации её возможностей.

## 1. Установка необходимых библиотек

Сначала установим необходимые зависимости для работы с LLaVA.

In [ ]:
!pip install torch torchvision transformers accelerate sentencepiece protobuf==3.20.3 gradio
!pip install git+https://github.com/huggingface/transformers

## 2. Клонирование репозитория LLaVA

Клонируем официальный репозиторий LLaVA для использования его функциональности.

In [ ]:
!git clone https://github.com/haotian-liu/LLaVA.git
%cd LLaVA
!pip install -e .

## 3. Загрузка модели LLaVA

Загрузим предобученную модель LLaVA с Hugging Face. Мы будем использовать версию LLaVA-1.5, которая является улучшенной версией оригинальной модели.

In [ ]:
import torch
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model
from llava.conversation import conv_templates
from llava.utils import disable_torch_init
from PIL import Image
import requests
from io import BytesIO

# Отключаем инициализацию torch для ускорения загрузки
disable_torch_init()

# Загружаем модель LLaVA-1.5 7B (меньшая версия для быстрой работы в Colab)
model_path = "liuhaotian/llava-v1.5-7b"
model_name = get_model_name_from_path(model_path)
tokenizer, model, processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name=model_name
)

print("Модель LLaVA успешно загружена!")
print(f"Имя модели: {model_name}")

## 4. Функция для обработки изображений и генерации ответов

Создадим функцию, которая будет принимать изображение и вопрос, а затем генерировать ответ с помощью модели LLaVA.

In [ ]:
def process_image_and_generate_response(image, prompt, temperature=0.2, max_new_tokens=512):
    """
    Обрабатывает изображение и генерирует текстовый ответ на основе заданного вопроса.
    
    Args:
        image: PIL изображение или URL изображения
        prompt: Текстовый вопрос или инструкция
        temperature: Параметр температуры для генерации текста (0.0-1.0)
        max_new_tokens: Максимальное количество новых токенов для генерации
        
    Returns:
        Текстовый ответ модели
    """
    # Если передан URL, загружаем изображение
    if isinstance(image, str) and (image.startswith('http://') or image.startswith('https://')):
        response = requests.get(image)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    elif not isinstance(image, Image.Image):
        image = Image.open(image).convert('RGB')
    
    # Подготавливаем шаблон разговора
    # Исправление: используем 'vicuna' вместо model_name в качестве ключа
    conv = conv_templates['vicuna'].copy()
    conv.append_message(conv.roles[0], prompt)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()
    
    # Обрабатываем изображение и генерируем ответ
    image_tensor = processor(image).unsqueeze(0).to(model.device)
    
    input_ids = tokenizer(prompt).input_ids
    input_ids = torch.tensor(input_ids).unsqueeze(0).to(model.device)
    
    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=image_tensor,
            do_sample=True,
            temperature=temperature,
            max_new_tokens=max_new_tokens
        )
    
    outputs = tokenizer.decode(output_ids[0, input_ids.shape[1]:], skip_special_tokens=True)
    return outputs

## 5. Создание интерактивного интерфейса с помощью Gradio

Создадим простой веб-интерфейс для взаимодействия с моделью LLaVA с помощью библиотеки Gradio.

In [ ]:
import gradio as gr

def llava_interface(image, prompt, temperature=0.2, max_tokens=512):
    if image is None:
        return "Пожалуйста, загрузите изображение."
    if not prompt:
        return "Пожалуйста, введите вопрос или инструкцию."
    
    try:
        response = process_image_and_generate_response(
            image, prompt, temperature=temperature, max_new_tokens=max_tokens
        )
        return response
    except Exception as e:
        return f"Произошла ошибка: {str(e)}"

# Создаем интерфейс
demo = gr.Interface(
    fn=llava_interface,
    inputs=[
        gr.Image(type="pil", label="Загрузите изображение"),
        gr.Textbox(lines=2, placeholder="Введите вопрос или инструкцию...", label="Вопрос/Инструкция"),
        gr.Slider(minimum=0.0, maximum=1.0, value=0.2, step=0.1, label="Температура"),
        gr.Slider(minimum=64, maximum=1024, value=512, step=64, label="Максимальное количество токенов")
    ],
    outputs=gr.Textbox(label="Ответ модели"),
    title="Демонстрация LLaVA: Крупная языковая и визуальная модель",
    description="Загрузите изображение и задайте вопрос или инструкцию. Модель LLaVA проанализирует изображение и сгенерирует текстовый ответ."
)

# Запускаем интерфейс
demo.launch(share=True, debug=True)

## 6. Примеры использования LLaVA

Давайте рассмотрим несколько примеров использования модели LLaVA с различными типами изображений и запросов.

In [ ]:
# Пример 1: Описание изображения
image_url = "https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg"
response = requests.get(image_url)
example_image = Image.open(BytesIO(response.content)).convert('RGB')
example_image.save('example_image.jpg')
example_image

In [ ]:
# Запрос на описание изображения
prompt = "Опиши подробно, что ты видишь на этом изображении."
response = process_image_and_generate_response(example_image, prompt)
print(response)

In [ ]:
# Пример 2: Пространственное рассуждение
prompt = "Что находится слева от человека на изображении? Опиши этот объект."
response = process_image_and_generate_response(example_image, prompt)
print(response)

In [ ]:
# Пример 3: Сложное рассуждение
prompt = "Предположи, какое время года изображено на фотографии и объясни, почему ты так считаешь."
response = process_image_and_generate_response(example_image, prompt)
print(response)

## 7. Загрузка собственного изображения

Вы можете загрузить собственное изображение и задать вопрос или инструкцию. Для этого используйте интерактивный интерфейс, созданный выше, или следующий код:

In [ ]:
from google.colab import files
import matplotlib.pyplot as plt

# Загрузка изображения
uploaded = files.upload()
image_path = list(uploaded.keys())[0]
user_image = Image.open(image_path).convert('RGB')

# Отображение изображения
plt.figure(figsize=(10, 10))
plt.imshow(user_image)
plt.axis('off')
plt.show()

# Ввод вопроса
user_prompt = input("Введите вопрос или инструкцию: ")

# Генерация ответа
response = process_image_and_generate_response(user_image, user_prompt)
print("\nОтвет модели:")
print(response)

## 8. Заключение

В этом ноутбуке мы продемонстрировали работу модели LLaVA, которая объединяет возможности обработки изображений и текста. Модель способна:

1. Детально описывать содержимое изображений
2. Отвечать на вопросы о пространственных отношениях объектов
3. Проводить сложные рассуждения на основе визуальной информации
4. Следовать инструкциям пользователя при анализе изображений

LLaVA представляет собой значительный шаг в создании универсальных мультимодальных ассистентов, способных понимать как текстовую, так и визуальную информацию.

### Ссылки и ресурсы

- [Официальный репозиторий LLaVA на GitHub](https://github.com/haotian-liu/LLaVA)
- [Статья LLaVA на NeurIPS 2023](https://arxiv.org/abs/2304.08485)
- [Модели LLaVA на Hugging Face](https://huggingface.co/liuhaotian/llava-v1.5-7b)